In [89]:
from wherobots.db import connect
from wherobots.db.region import Region
from wherobots.db.runtime import Runtime
import geopandas as gpd
from dotenv import load_dotenv
import os
from shapely import wkt

In [90]:
load_dotenv()
api_key = os.getenv("WHEROBOTS_API_KEY")

# Define Helper Functions

In [91]:
conn = None
curr = None

def open_wherobots_connection(api_key, host='api.cloud.wherobots.com', runtime=Runtime.SEDONA, region=Region.AWS_US_WEST_2):
    global conn, curr
    conn = connect(
        api_key=api_key,
        runtime=runtime,
        region=region,
        host=host
    )
    curr = conn.cursor()
    return conn, curr

def close_wherobots_connection():
    global conn, curr
    if curr is not None:
        curr.close()
    if conn is not None:
        conn.close()

def execute_wherobots_query(sql):
    global conn, curr
    if conn is None or curr is None:
        print("Connection is not open. Please open the connection before executing queries.")
        return
    curr.execute(sql)
    results = curr.fetchall()
    return results

def oneoff_wherobots_query(sql, api_key):
    with connect(api_key=api_key, runtime=Runtime.SEDONA, region=Region.AWS_US_WEST_2, host='api.cloud.wherobots.com') as conn:
        curr = conn.cursor()
        curr.execute(sql)
        results = curr.fetchall()
    return results

# Setup a couple SQL queries ahead of time for testing

In [92]:
sql_query_overture_countries = """
    SELECT
        id,
        names['primary'],
        geometry,
        population
    FROM
        wherobots_open_data.overture_2024_02_15.admins_locality
    WHERE localityType = 'country'
    SORT BY population DESC
    LIMIT 10
"""

In [93]:
sql_query_overture_places = """
    SELECT
        id, names.common[0].value AS name, ST_X(geometry) AS long, ST_Y(geometry) AS lat, geometry,
        CASE WHEN names.common[0].value LIKE "%Torchy's Tacos%" THEN "Torchy's Tacos" ELSE 'Velvet Taco' END AS chain
    FROM 
        wherobots_open_data.overture.places_place
    WHERE 
        addresses[0].region = 'TX'
        AND (names.common[0].value LIKE "%Torchy's Tacos%" OR names.common[0].value LIKE '%Velvet Taco%')
"""

# Open the Wherobots connection
## Remember to close this when done using the `close_wherobots_connection()` function

In [94]:
open_wherobots_connection(api_key)

(<wherobots.db.connection.Connection at 0x16da3d410>,
 <wherobots.db.cursor.Cursor at 0x16be05790>)

In [95]:
execute_wherobots_query("SHOW SCHEMAS in wherobots_pro_data")

,namespace
0,osm
1,us_census
2,weather
3,google_microsoft
4,nyc_taxi
5,landsat


In [96]:
execute_wherobots_query("SHOW TABLES in wherobots_pro_data.us_census")

,namespace,tableName,isTemporary
0,us_census,zipcode,False


In [97]:
execute_wherobots_query(sql_query_overture_countries)

,id,names.primary,geometry,population
0,0850c861bfffffff01aeb407d56d3441,United States,POINT (-100.445882 39.7837304),331449281
1,0851f1eaffffffff0164d4bb993451b0,Brasil,POINT (-53.2 -10.3333333),208714806
2,0858e2a17fffffff0172bf8a197f4f8a,Nigeria,POINT (7.9999721 9.6000359),152217341
3,085ef259bfffffff01163555bf80130a,México,POINT (-102.0077097 23.6585116),129900000
4,085d36903fffffff01a647761fcfdaf3,Abyssinia,POINT (38.6521203 10.2116702),126527060
5,085eea613fffffff01a8a15aa3da4c44,United Kingdom,POINT (-3.2765753 54.7023545),61792000
6,08543a95bfffffff011612e1eb2b785d,Argentina,POINT (-64.9672817 -34.9964963),47327407
7,0850301d3fffffff01fc58a07e8605b4,España,POINT (-4.8379791 39.3260685),46157822
8,085de865bfffffff0135578029abeb35,Colombia,POINT (-72.9088133 4.099917),44050548
9,0850b2873fffffff01024304ec4f908c,Sudan,POINT (29.4917691 14.5844444),39379358


In [98]:
df = execute_wherobots_query(sql_query_overture_places)
df['geometry'] = df['geometry'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, geometry = 'geometry')
gdf = gdf.set_crs(epsg=4326)
gdf

,id,name,long,lat,geometry,chain
0,tmp_8104A792162544C5EC33BF66A9B4456E,Torchy's Tacos,-98.596890,29.608910,POINT (-98.59689 29.60891),Torchy's Tacos
1,tmp_D17CA8BD72325496F228D1A974EBB298,Torchy's Tacos,-97.741750,30.293680,POINT (-97.74175 30.29368),Torchy's Tacos
2,tmp_F497329382C10CA08FD8ECAEB2B9E561,Velvet Taco,-95.488660,30.183140,POINT (-95.48866 30.18314),Velvet Taco
3,tmp_9B40A1BF3237E63DFEC597344A041BF9,Torchy's Tacos,-96.805853,32.909982,POINT (-96.80585 32.90998),Torchy's Tacos
4,tmp_38210E5EC047B8B834203756B5F2602C,Torchy's Tacos,-96.687550,33.101180,POINT (-96.68755 33.10118),Torchy's Tacos
...,...,...,...,...,...,...
104,tmp_AFBED999C7BCE9E988C0CB40845C45F3,Torchy's Tacos,-101.868980,33.586880,POINT (-101.86898 33.58688),Torchy's Tacos
105,tmp_338758358E507681C88793730AD85C80,Torchy's Tacos,-101.945018,33.565217,POINT (-101.94502 33.56522),Torchy's Tacos
106,tmp_F77A7A58172D7706BA11000071B01806,Torchy's Tacos,-102.367138,31.849462,POINT (-102.36714 31.84946),Torchy's Tacos
107,tmp_290EF25E70FE1610F4968B8DA88277BE,Velvet Taco,-101.868720,33.588017,POINT (-101.86872 33.58802),Velvet Taco


In [99]:
gdf.explore()

In [100]:
close_wherobots_connection()

# One-off query that opens and closes the connection for you

In [101]:
oneoff = oneoff_wherobots_query(sql_query_overture_places, api_key)

In [102]:
oneoff['geometry'] = oneoff['geometry'].apply(wkt.loads)

oneoff_gdf = gpd.GeoDataFrame(oneoff, geometry = 'geometry')
oneoff_gdf = oneoff_gdf.set_crs(epsg=4326)
oneoff_gdf

,id,name,long,lat,geometry,chain
0,tmp_8104A792162544C5EC33BF66A9B4456E,Torchy's Tacos,-98.596890,29.608910,POINT (-98.59689 29.60891),Torchy's Tacos
1,tmp_D17CA8BD72325496F228D1A974EBB298,Torchy's Tacos,-97.741750,30.293680,POINT (-97.74175 30.29368),Torchy's Tacos
2,tmp_F497329382C10CA08FD8ECAEB2B9E561,Velvet Taco,-95.488660,30.183140,POINT (-95.48866 30.18314),Velvet Taco
3,tmp_9B40A1BF3237E63DFEC597344A041BF9,Torchy's Tacos,-96.805853,32.909982,POINT (-96.80585 32.90998),Torchy's Tacos
4,tmp_38210E5EC047B8B834203756B5F2602C,Torchy's Tacos,-96.687550,33.101180,POINT (-96.68755 33.10118),Torchy's Tacos
...,...,...,...,...,...,...
104,tmp_AFBED999C7BCE9E988C0CB40845C45F3,Torchy's Tacos,-101.868980,33.586880,POINT (-101.86898 33.58688),Torchy's Tacos
105,tmp_338758358E507681C88793730AD85C80,Torchy's Tacos,-101.945018,33.565217,POINT (-101.94502 33.56522),Torchy's Tacos
106,tmp_F77A7A58172D7706BA11000071B01806,Torchy's Tacos,-102.367138,31.849462,POINT (-102.36714 31.84946),Torchy's Tacos
107,tmp_290EF25E70FE1610F4968B8DA88277BE,Velvet Taco,-101.868720,33.588017,POINT (-101.86872 33.58802),Velvet Taco


In [103]:
oneoff_gdf.explore()